In [7]:
import pandas as pd
import os

# 월별 CD금리 파일 불러오기
cd_df = pd.read_csv('./csv/월별_CD금리.csv')


# 펀드 파일 전처리
# 경로설정
path = './excel/주식형펀드/'

# 빈 리스트 생성
file_list = []

# 경로 내 모든 xls파일을 file_list에 저장
for filename in os.listdir(path):
    if filename.endswith('.xls'):
        file_list.append(filename)

# 데이터 프레임 생성
full_df = pd.DataFrame()
full_df["펀드명"] = ""
full_df["운용규모"] = ""
full_df["샤프지수"] = ""
full_df["KOSPI 샤프지수"] = ""
full_df["소르티노지수"] = ""
full_df["KOSPI 소르티노지수"] = ""

# 리스트에 저장된 파일 하나씩 불러오기
for file in file_list:
    # print(file)
    name = file.split('_')[0]
    fund_df = pd.read_excel(path+file)

    # 필요한 컬럼만 남긴다
    # 기준일자, 기준가격, 전일대비\n등락, 벤치마크지수
    fund_df = fund_df[["기준일자", "기준가격", "벤치마크지수"]]

    # 컬럼명 변경
    fund_df.rename(columns={"벤치마크지수" : "KOSPI"}, inplace=True)
    
    # 불필요 행 제거
    fund_df = fund_df.loc[1:]
    
    # 기준일자 오름차순으로 변경
    fund_df.sort_values("기준일자", ascending=True, inplace=True)    
    
    # 인덱스 재설정
    fund_df.reset_index(drop=True, inplace=True)    
    
    # 기준월 컬럼 생성
    fund_df["기준일자"] = fund_df["기준일자"].str.replace("/", "")
    fund_df["기준월"] = fund_df["기준일자"].astype(str).str[:6]     
    
    # 결측치 제거
    fund_df.dropna(inplace=True)
    fund_df.reset_index(drop=True, inplace=True)
    
    # 기준일자별 마지막 날짜
    monthly_last_day = fund_df.groupby(["기준월"])["기준일자"].max()
    monthly_last_day = pd.DataFrame(monthly_last_day)    

    # 샤프지수용 데이터 프레임 merge    
    sha_df = pd.merge(monthly_last_day, fund_df, how="left", on="기준일자")     
        
    # 샤프지수용 월별수익률 컬럼 생성
    sha_df["월별수익률"] = sha_df["기준가격"].pct_change()
    
    # 샤프지수용 KOSPI 월별수익률 컬럼 생성
    sha_df["KOSPI 월별수익률"] = sha_df["KOSPI"].pct_change()

    sha_df.dropna(inplace=True)
    sha_df.reset_index(drop=True, inplace=True)
    
    # 샤프지수용 월별수익률을 사용한 3년 수익률 표준편차
    sha_year3_std = sha_df["월별수익률"].std(ddof=0)
    sha_df["3년 표준편차"] = sha_year3_std

    # 샤프지수용 KOSPI 월별수익률을 사용한 KOSPI 3년 수익률 표준편차
    sha_kospi_year3_std = sha_df["KOSPI 월별수익률"].std(ddof=0)
    sha_df["KOSPI 3년 표준편차"] = sha_kospi_year3_std

    # 샤프지수 데이터 프레임
    sha_df = sha_df[["기준일자", "기준가격", "KOSPI", "기준월", "월별수익률", "3년 표준편차", "KOSPI 월별수익률", "KOSPI 3년 표준편차"]]
    
    # 소르티노지수용 데이터 프레임 merge    
    sor_df = pd.merge(monthly_last_day, fund_df, how="left", on="기준일자")

    # 소르티노지수용 월별수익률 컬럼 생성
    sor_df["월별수익률"] = sor_df["기준가격"].pct_change()
    
    # 소르티노지수용 KOSPI 월별수익률 컬럼 생성
    sor_df["KOSPI 월별수익률"] = sor_df["KOSPI"].pct_change()

    # 소르티노지수용 월별수익률이 0 이하인 값만 남긴다
    negative_sor_df = sor_df[sor_df["월별수익률"] < 0]
    negative_sor_df.dropna(inplace=True)
    negative_sor_df.reset_index(drop=True, inplace=True)
    
    # 소르티노지수용 KOSPI 월별수익률이 0 이하인 값만 남긴다
    kospi_sor_df = sor_df[sor_df["KOSPI 월별수익률"] < 0]
    kospi_sor_df.dropna(inplace=True)
    kospi_sor_df.reset_index(drop=True, inplace=True)
    
    negative_sor_df["KOSPI 월별수익률"] = kospi_sor_df["KOSPI 월별수익률"]
    
    # 소르티노지수용 월별수익률을 사용한 3년 수익률 표준편차
    sor_year3_std = negative_sor_df["월별수익률"].std(ddof=0)
    sor_df["3년 표준편차"] = sor_year3_std

    # 소르티노지수용 KOSPI 월별수익률을 사용한 KOSPI 3년 수익률 표준편차
    sor_kospi_year3_std = negative_sor_df["KOSPI 월별수익률"].std(ddof=0)
    sor_df["KOSPI 3년 표준편차"] = sor_kospi_year3_std
    sor_df.dropna(inplace=True)
    sor_df.reset_index(drop=True, inplace=True)

    # 소르티노지수 데이터 프레임
    sor_df = sor_df[["기준일자", "기준가격", "KOSPI", "기준월", "월별수익률", "3년 표준편차", "KOSPI 월별수익률", "KOSPI 3년 표준편차"]]

    # CD금리 전처리
    # df 복사
    sha_fund_cd_df = sha_df.copy()
    sor_fund_cd_df = sor_df.copy()

    # CD금리 컬럼 추가
    sha_fund_cd_df["CD금리"] = cd_df["원자료"]
    sor_fund_cd_df["CD금리"] = cd_df["원자료"]
    
    # CD금리 평균
    sha_fund_cd_df["CD금리"] = sha_fund_cd_df["CD금리"].mean()
    sor_fund_cd_df["CD금리"] = sor_fund_cd_df["CD금리"].mean()

    sha_fund_cd_df["CD금리"] = sha_fund_cd_df["CD금리"]/100
    sha_fund_cd_df.dropna(inplace=True)

    sor_fund_cd_df["CD금리"] = sor_fund_cd_df["CD금리"]/100
    sor_fund_cd_df.dropna(inplace=True)

    # 샤프지수 초과수익률 계산
    sha_excess_return = ((sha_fund_cd_df["기준가격"].iloc[-1]-sha_fund_cd_df["기준가격"].iloc[0])/sha_fund_cd_df["기준가격"].iloc[0])-sha_fund_cd_df["CD금리"][0]
    sha_kospi_excess_return = ((sha_fund_cd_df["KOSPI"].iloc[-1]-sha_fund_cd_df["KOSPI"].iloc[0])/sha_fund_cd_df["KOSPI"].iloc[0])-sha_fund_cd_df["CD금리"][0]
    
    sha_return_std = sha_fund_cd_df["3년 표준편차"][0]
    sha_kospi_return_std = sha_fund_cd_df["KOSPI 3년 표준편차"][0]
    
    # 샤프지수 데이터 프레임 생성 및 컬럼 추가
    sharpe_df = pd.DataFrame()
    sharpe_df["기준월"] = sha_fund_cd_df["기준월"]
    sharpe_df["종가"] = sha_fund_cd_df["기준가격"]
    sharpe_df["KOSPI 종가"] = sha_fund_cd_df["KOSPI"]
    sharpe_df["초과수익률"] = sha_excess_return
    sharpe_df["3년 표준편차"] = sha_return_std
    sharpe_df["KOSPI 초과수익률"] = sha_kospi_excess_return
    sharpe_df["KOSPI 3년 표준편차"] = sha_kospi_return_std
    
    # 샤프지수 계산    
    sharpe_df["샤프지수"] = ""
    sharpe_df["샤프지수"] = sharpe_df["초과수익률"]/sharpe_df["3년 표준편차"]

    sharpe_df["KOSPI 샤프지수"] = ""
    sharpe_df["KOSPI 샤프지수"] = sharpe_df["KOSPI 초과수익률"]/sharpe_df["KOSPI 3년 표준편차"]
    
    sharpe_df["CD금리"] = sha_fund_cd_df["CD금리"]

    # 소르티노지수 초과수익률 및 표준편차 계산
    sor_excess_return = ((sor_fund_cd_df["기준가격"].iloc[-1]-sor_fund_cd_df["기준가격"].iloc[0])/sor_fund_cd_df["기준가격"].iloc[0])-sor_fund_cd_df["CD금리"][0]
    sor_kospi_excess_return = ((sor_fund_cd_df["KOSPI"].iloc[-1]-sor_fund_cd_df["KOSPI"].iloc[0])/sor_fund_cd_df["KOSPI"].iloc[0])-sor_fund_cd_df["CD금리"][0]
    sor_return_std = sor_fund_cd_df["3년 표준편차"][0]
    sor_kospi_return_std = sor_fund_cd_df["KOSPI 3년 표준편차"][0]

    # 소르티노지수 데이터 프레임 생성 및 컬럼 추가
    sortino_df = pd.DataFrame()
    sortino_df["기준월"] = sor_fund_cd_df["기준월"]
    sortino_df["종가"] = sor_fund_cd_df["기준가격"]
    sortino_df["KOSPI 종가"] = sor_fund_cd_df["KOSPI"]
    sortino_df["초과수익률"] = sor_excess_return
    sortino_df["3년 표준편차"] = sor_return_std
    sortino_df["KOSPI 초과수익률"] = sor_kospi_excess_return
    sortino_df["KOSPI 3년 표준편차"] = sor_kospi_return_std

    # 소르티노지수 계산    
    sortino_df["소르티노지수"] = ""
    sortino_df["소르티노지수"] = sortino_df["초과수익률"]/sortino_df["3년 표준편차"]
    
    sortino_df["KOSPI 소르티노지수"] = ""
    sortino_df["KOSPI 소르티노지수"] = sortino_df["KOSPI 초과수익률"]/sortino_df["KOSPI 3년 표준편차"]
    
    sortino_df["CD금리"] = sor_fund_cd_df["CD금리"]
    
    sharpe_sortino_df = pd.merge(sharpe_df, sortino_df, how="left", on="기준월")
        
    fund_sharpe_sortino_df = sharpe_sortino_df.loc[[0], ["샤프지수", "KOSPI 샤프지수", "소르티노지수", "KOSPI 소르티노지수"]]

    full_df.reset_index(drop=True, inplace=True)
    full_df = pd.concat([full_df, fund_sharpe_sortino_df])

# 파일에서 확장자와 '_'를 제외한 파일명만 리스트화
file_name_list=[]
for file in os.listdir(path):     
    name = file.split('_')[0]
    file_name_list.append(name)

# full_df에 펀드명 추가
full_df["펀드명"] = file_name_list

C:\Users\user\AppData\Local\Temp\ipykernel_16620\244679663.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_sor_df.dropna(inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_16620\244679663.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kospi_sor_df.dropna(inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_16620\244679663.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [17]:
fund_scale_df = pd.read_excel('./excel/주식형운용규모.xlsx')
full_df["운용규모"] = fund_scale_df["운용규모"]
full_df["위험등급"] = fund_scale_df["위험등급"]
full_df.reset_index(drop=True, inplace=True)

In [18]:
full_df["위험등급"] = full_df["위험등급"].str.replace("\(UH\)", "")

C:\Users\user\AppData\Local\Temp\ipykernel_16620\2366646860.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  full_df["위험등급"] = full_df["위험등급"].str.replace("\(UH\)", "")


In [12]:
# csv 파일로 저장
full_df.to_csv('./csv/주식형펀드_샤프지수_소르티노지수.csv', encoding='CP949')